### Importing libraries ###

In [6]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [52]:
import torchtext

In [51]:
%pip install torchtext
from torchtext.vocab import GloVe;
global_vectors = GloVe(name='6B', dim=300);

Note: you may need to restart the kernel to use updated packages.


.vector_cache\glove.6B.zip: 862MB [03:09, 4.54MB/s]                               
100%|█████████▉| 399999/400000 [00:55<00:00, 7198.30it/s]


In [ ]:
import re
import torchtext.vocab as tv


In [23]:
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import re
from argparse import Namespace
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import sentencepiece as spm


In [56]:
from argparse import Namespace

args = Namespace(
data_directory = "../data/toxic_comments_sample.csv",
seed = 1300,
max_sentence_length = 100,
lower = True,
start_token = "<sos>",
end_token = "<eos>",
unk_token = "<unk>",
pad_token = "<pad>",
vocab_size = 0,
embedding_dim = 300,
device = "cuda" if torch.cuda.is_available() else "cpu",
);

In [60]:
toxic_frame = pd.read_csv(args.data_directory)
toxic_frame

,Unnamed: 0,id,comment_text,target
0,0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0
...,...,...,...,...
2995,2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0
2996,2996,081166fea250a5af,and asking top stop involving me,0.0
2997,2997,08121cde0a727842,re all items i know that you said i did someth...,0.0
2998,2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0


In [61]:
train_set_len = int(len(toxic_frame)*0.80);

test_set_len = int(len(toxic_frame)*0.10);

val_set_len = int(len(toxic_frame)*0.10);

toxic_frame.loc[:train_set_len,'split'] = "train";

toxic_frame.loc[train_set_len:train_set_len + test_set_len,'split'] = "test";

toxic_frame.loc[train_set_len + test_set_len:,'split'] = "val";

toxic_frame.to_csv(f"../data/data_split.csv");

args.data_split = "../data/data_split.csv";

In [62]:
toxic_frame

,Unnamed: 0,id,comment_text,target,split
0,0,0000997932d777bf,explanation why the edits made under my userna...,0.0,train
1,1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0,train
2,2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0,train
3,3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0,train
4,4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0,train
...,...,...,...,...,...
2995,2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0,val
2996,2996,081166fea250a5af,and asking top stop involving me,0.0,val
2997,2997,08121cde0a727842,re all items i know that you said i did someth...,0.0,val
2998,2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0,val


In [ ]:
import torch
from torch.utils.data import Dataset
def pad_sequence(tokenizer,encoded_ids, max_length, pad_id = args.pad_token):
    
    # Truncate if too long
    if len(encoded_ids) > max_length:
        encoded_ids = encoded_ids[:max_length]
    
    # Add padding
    padding = [pad_id] * (max_length - len(encoded_ids))
    return encoded_ids + padding;

In [110]:
class Vectorizer():
    
    def __init__(self,tokenizer,vocabulary):
        self.tokenizer = tokenizer;
        self.vocabulary = vocabulary;
        pass;
    
    @classmethod
    def from_dataframe(cls, csvFile,columns):
        """
        Instantiate the vectorizer from the dataset dataframe
        
        Args:

        dataFrame (pandas.DataFrame): Dataset 
        xColumnName : name of features column.

        Returns: an instance of the Vectorizer

        """

        datafile = pd.read_csv(csvFile);

        tokens_iterator = [];

        tokenizer = torchtext.data.utils.get_tokenizer("basic_english");
        
        for i,row in datafile[[columns[0]]].iterrows():
            text = row[columns[0]];
            tokens = tokenizer(text);
            tokens_iterator.append(tokens);

        caption_vocab = tv.build_vocab_from_iterator(tokens_iterator,min_freq = 3,specials=[args.pad_token, args.unk_token, args.start_token , args.end_token]);
        
        caption_vocab.set_default_index(caption_vocab[args.unk_token]);
   
        return cls(tokenizer,caption_vocab);
    
    
    
    def vectorize(self,sentence , max_length , add_special = False):
        
        """
        Create a numerical vector for the sentence
        
        Args:
        tokenizer: sentencepiece tokenizer.
        sentence (str): the sentence
        add_special (boolean) : Adding special tokens (<sos>,<eos>).
        
        Returns:
        Tokens indices vector.
        
        """
        vector = self.tokenizer(sentence);
        vector = pad_sequence(self.tokenizer,vector, max_length);
        ids = [];
        for i,token in enumerate(vector):
            ids.append(self.vocabulary[token]);
        
        return ids;


In [111]:
class CustomDataSet(Dataset):
    
    def __init__(self,datafile,vectorizer):
        
        dataFrame = pd.read_csv(datafile);
        
        self._dataFrame = dataFrame;
        self._vectorizer = vectorizer;
        
        self.train_df = self._dataFrame[self._dataFrame.split == "train"];
        self.train_size = len(self.train_df);
        
        self.val_df = self._dataFrame[self._dataFrame.split == "val"];
        self.val_size = len(self.val_df);
        
        self.test_df = self._dataFrame[self._dataFrame.split == "test"];
        self.test_size = len(self.test_df);

        self._lookup_dict = {'train': (self.train_df, self.train_size),
        'val': (self.val_df, self.val_size),
        'test': (self.test_df, self.test_size)};
        
        self.set_split();  # setting train as default data set.

    
    @classmethod
    def load_dataset_and_make_vectorizer(cls,datafile):
        datafile = datafile;
        return cls(datafile, Vectorizer.from_dataframe(datafile,['comment_text']));

    
    def get_vectorizer(self):
        return self._vectorizer;
    
    
    def set_split(self, split="train"):
        
        """ selects the splits in the dataset using a column in the dataframe
        Args:finaldata.csv
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split;

        self._target_df, self._target_size = self._lookup_dict[split];
    
    

    def __getitem__(self, index):
        
        """
        the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        image,target tensors + image name for testing purpose.
        """
        row = self._target_df.iloc[index];        
        
        source = self._vectorizer.vectorize(row['comment_text'],args.max_sentence_length);
        
        target = row['target'];

        return torch.tensor(source),torch.tensor(target);
    
    def __len__(self):
        return self._target_size;


    

In [112]:
dataset = CustomDataSet.load_dataset_and_make_vectorizer(args.data_split)
dataloader = DataLoader(dataset,batch_size=20);


In [113]:
for x,y in dataloader:
    break;
print(x,y)

tensor([[ 660,   81,    4,  ...,    0,    0,    0],
        [   1,   59, 1472,  ...,    0,    0,    0],
        [ 525,  383,   70,  ...,    0,    0,    0],
        ...,
        [ 432,    1, 2908,  ...,    0,    0,    0],
        [   4,    1,  118,  ...,    0,    0,    0],
        [  57,  233,    5,  ...,    0,    0,    0]]) tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0.])


In [114]:
import numpy as np

In [125]:
import torch
import torch.nn as nn

def prepare_embedding_matrix(dataset_words, pre_embeddings):
    word_to_idx = pre_embeddings.stoi
    embedding_matrix = pre_embeddings.vectors
    embedding_dim = embedding_matrix.shape[1]

    final_embeddings = torch.zeros((len(dataset_words), embedding_dim))

    for i, word in enumerate(dataset_words):
        if word in word_to_idx and word_to_idx[word] < embedding_matrix.shape[0]:
            final_embeddings[i] = embedding_matrix[word_to_idx[word]]
        else:
            embedding_i = torch.empty(embedding_dim)
            torch.nn.init.xavier_uniform_(embedding_i.unsqueeze(0))
            final_embeddings[i] = embedding_i

    embedding_layer = nn.Embedding(
        num_embeddings=len(dataset_words),
        embedding_dim=embedding_dim
    )
    embedding_layer.weight.data = final_embeddings

    return embedding_layer


In [126]:
embedding_layer = prepare_embedding_matrix(list(dataset.get_vectorizer().vocabulary.get_stoi().keys()),global_vectors);